# Neural Network - Abalone Age by Classification

# Project 3 - Predicting the Age of Abalone

In [1]:
# Dependancies

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense

## Data Preprocessing

In [2]:
# Read in unscaled version of the dataset
unscaled_df = pd.read_csv('./Dataset/abalone_unscaled_data.csv')
unscaled_df.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5


In [3]:
# Label Encode the categorical "Sex" column
# Note: Female => 0
#       Infant => 1
#       Male   => 2
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
unscaled_df['Sex_LabelEncoded'] = le.fit_transform(unscaled_df['Sex'])
unscaled_df.head()


,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded
0,M,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2
1,M,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2
2,F,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0
3,M,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2
4,I,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1


In [4]:
# Binary encode the categorical "Sex" column
unscaled_df = pd.get_dummies(unscaled_df, prefix=['Sex'], columns=['Sex'])
unscaled_df.head()

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age,Sex_LabelEncoded,Sex_F,Sex_I,Sex_M
0,91,73,19,102.8,44.9,20.2,30.0,15,16.5,2,0,0,1
1,70,53,18,45.1,19.9,9.7,14.0,7,8.5,2,0,0,1
2,106,84,27,135.4,51.3,28.3,42.0,9,10.5,0,1,0,0
3,88,73,25,103.2,43.1,22.8,31.0,10,11.5,2,0,0,1
4,66,51,16,41.0,17.9,7.9,11.0,7,8.5,1,0,1,0


In [5]:
# Reorganize columns
column_order = ["Sex_LabelEncoded", "Sex_M", "Sex_F", "Sex_I", "Length", 
                "Diameter", "Height", "Whole_weight", "Shucked_weight", 
                "Viscera_weight", "Shell_weight", "Rings", "Age"]
unscaled_df = unscaled_df.reindex(columns=column_order)
unscaled_df.head()

,Sex_LabelEncoded,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings,Age
0,2,1,0,0,91,73,19,102.8,44.9,20.2,30.0,15,16.5
1,2,1,0,0,70,53,18,45.1,19.9,9.7,14.0,7,8.5
2,0,0,1,0,106,84,27,135.4,51.3,28.3,42.0,9,10.5
3,2,1,0,0,88,73,25,103.2,43.1,22.8,31.0,10,11.5
4,1,0,0,1,66,51,16,41.0,17.9,7.9,11.0,7,8.5


## Determine X and y (Example using Sex_LabelEncoded and y=Rings)

In [6]:
#For the neural network classification, the categories must have one-hot encoding, not just label encoding
#That is why only the one-hot encoded columns for sex were included and not the other versions

# Determine X 
X = unscaled_df[['Sex_M', 'Sex_F', 'Sex_I', 'Length', 'Diameter', 'Height', 'Whole_weight', 
                 'Shucked_weight', 'Viscera_weight', 'Shell_weight']]
X

,Sex_M,Sex_F,Sex_I,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
0,1,0,0,91,73,19,102.8,44.9,20.2,30.0
1,1,0,0,70,53,18,45.1,19.9,9.7,14.0
2,0,1,0,106,84,27,135.4,51.3,28.3,42.0
3,1,0,0,88,73,25,103.2,43.1,22.8,31.0
4,0,0,1,66,51,16,41.0,17.9,7.9,11.0
...,...,...,...,...,...,...,...,...,...,...
4172,0,1,0,113,90,33,177.4,74.0,47.8,49.8
4173,1,0,0,118,88,27,193.2,87.8,42.9,52.1
4174,1,0,0,120,95,41,235.2,105.1,57.5,61.6
4175,0,1,0,125,97,30,218.9,106.2,52.2,59.2


## Prepare Data for Stratification

In [7]:
#The one-hot encoding for the rings is done in later cells
# Here we are determining the classes with only one value and removing them so that stratification on the train-test split will work
# Stratification is necessary because measureing the training v testing requires the same number of classes

# Determine y
y = unscaled_df["Rings"]
y

0       15
1        7
2        9
3       10
4        7
        ..
4172    11
4173    10
4174     9
4175    10
4176    12
Name: Rings, Length: 4177, dtype: int64

In [8]:
y.max()

29

In [9]:
y.min()

1

In [10]:
pd.Series(y).unique().size

28

## Turn Number of Rings into Bins

In [12]:
a = np.array(y)
unique, counts = np.unique(a, return_counts=True)
tally = dict(zip(unique, counts))
tally

{1: 1,
 2: 1,
 3: 15,
 4: 57,
 5: 115,
 6: 259,
 7: 391,
 8: 568,
 9: 689,
 10: 634,
 11: 487,
 12: 267,
 13: 203,
 14: 126,
 15: 103,
 16: 67,
 17: 58,
 18: 42,
 19: 32,
 20: 26,
 21: 14,
 22: 6,
 23: 9,
 24: 2,
 25: 1,
 26: 1,
 27: 2,
 29: 1}

In [13]:
# This is to record the classes that only have a single value
# Classes with single value have to be removed in order to do stratification

solo_classes=[];

for key in tally:
    if tally[key] == 1:
        solo_classes.append(key)
solo_classes

[1, 2, 25, 26, 29]

In [14]:
# This removes the rows with the classes with the single values for both X and y
solo_indices=[];
for i in range(len(solo_classes)):
    print(i)
    j = pd.Series(y)[pd.Series(y) == solo_classes[i]].index
    print(j)
    solo_indices.append(j)

0
Int64Index([236], dtype='int64')
1
Int64Index([719], dtype='int64')
2
Int64Index([2201], dtype='int64')
3
Int64Index([294], dtype='int64')
4
Int64Index([480], dtype='int64')


In [15]:
X.shape

(4177, 10)

In [16]:
for i in range(len(solo_indices)):
    print(i)
    X = X.drop(solo_indices[i])

0
1
2
3
4


In [17]:
X.shape

(4172, 10)

In [18]:
y.shape

(4177,)

In [19]:
for i in range(len(solo_indices)):
    print(i)
    y = y.drop(solo_indices[i])

0
1
2
3
4


In [20]:
y.shape

(4172,)

## Perform Train-Test-Split

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical

#NOTE: -1 after y is needed to correct for zero-indexing (the zero column)
# You would never have a training value for the zero column and it would only add noise.

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y-1)
encoded_y = label_encoder.transform(y-1)

# Step 2: Convert encoded labels to one-hot-encoding
y_categorical = to_categorical(encoded_y-1)

# Stratification is necessary to ensure that the test and training data are directly comparable
X_train, X_test, y_train, y_test = train_test_split(
    X, y_categorical, stratify=y, random_state=1)

In [22]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Scale our data

In [23]:
# Using MinMaxScaler instead of StandardScaler because this is a classification problem with a limited range
# Sex columns are already one-hot encoded

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Build a Deep Neural Network

In [24]:
# Create an empty sequential model
model = Sequential()

In [25]:
# The input layer has 10 columns
X_train_scaled.shape

(3129, 10)

In [26]:
# Add the first layer where the input dimensions are the 10 columns of the training data
model.add(Dense(100, activation='relu', input_dim=X_train.shape[1]))

In [27]:
# Add a second layer, which is the first hidden layer
model.add(Dense(1000, activation='relu'))

In [28]:
# Add a second hidden layer
model.add(Dense(1000, activation='relu'))

In [29]:
# Add a third hidden layer
model.add(Dense(1000, activation='relu'))

In [30]:
# The output layer has 22 columns that are one-hot encoded
y_train.shape

(3129, 22)

In [31]:
# Add output layer
model.add(Dense(y_train.shape[1], activation="softmax"))

In [32]:
# Compile the model using categorical_crossentropy for the loss function, the adam optimizer,
# and add accuracy to the training metrics
model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=['accuracy'])

In [33]:
# Use the training data to fit (train) the model
model.fit(
    X_train_scaled,
    y_train,
    epochs=2000,
    shuffle=True,
    verbose=2
)

Epoch 1/2
98/98 - 1s - loss: 2.2931 - accuracy: 0.2125
Epoch 2/2
98/98 - 2s - loss: 2.1064 - accuracy: 0.2438


# Save the Trained Model

In [34]:
# Save the model
model.save("Abalone_trained_NNClass_NoBin_v5.h5")

# Evaluate the Model

In [35]:
# Load the model
from tensorflow.keras.models import load_model
model = load_model("Abalone_trained_NNClass_NoBin_v5.h5")

In [36]:
y_test.shape

(1043, 22)

In [37]:
# Evaluate the model using the training data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

33/33 - 0s - loss: 2.0455 - accuracy: 0.2627
Loss: 2.04551362991333, Accuracy: 0.2627037465572357


In [38]:
# Grab just one data point to test with
test = np.expand_dims(X_test_scaled[0], axis=0)
test.shape

(1, 10)

In [39]:
# Make a prediction. The result should be 5 - STANDING
print(f"Predicted class: {model.predict_classes(test)}")

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
Predicted class: [3]
